<a href="https://colab.research.google.com/github/juneilsam/PUBG/blob/main/PUBG_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**대상**

- STEAM

- 2018-01 시즌

- SOLO

- Korea˙Japan, Asia

개인정보 노출을 우려하여 출력 결과는 포함하지 않았습니다.

In [ ]:
# 주요 모듈 불러오기

import requests                         # 웹페이지 접속
import json                             # json 데이터 가공 
import time                             # 분당 접속 제한
from tqdm import tqdm                   # 진행 상황 확인
import csv                              # csv 파일로 변환
import pandas as pd                     # dataframe으로의 가공
from google.colab import drive          # csv 파일 저장 경로 지정
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Authorization key를 이용한 접속 허가

header = {
  "Authorization": "KEY",
  "Accept": "application/vnd.api+json"
}

In [ ]:
# leaderboards에서 상위권 players' id 수집하기
player_list = []  # players' id 리스트

def leaders(area):

    r = requests.get(f"https://api.pubg.com/shards/pc-{area}/leaderboards/division.bro.official.pc-2018-10/solo", headers=header)

    player_Id = json.loads(r.text)

    p = player_Id['data']['relationships']['players']['data']

    for n in p:
        player_list.append(n.get("id"))

In [ ]:
for area in ['krjp', 'as']:
  leaders(area)

In [ ]:
print(player_list) # 수집된 데이터 확인
print(len(player_list)) # 인원 수

In [ ]:
# matchId 수집하기

match_list = []

def gathering_match(player):

    url = f"https://api.pubg.com/shards/steam/players/{player}"

    r = requests.get(url, headers=header)

    player_stat = json.loads(r.text)
    
    g = player_stat['data']['relationships']['matches']['data']  # 각 매치 데이터

    for n in g:
      match_list.append(n.get("id"))  # 각 매치 별 id 선별

In [ ]:
time.sleep(60) # 분당 접속 제한을 피하기 위함

In [ ]:
# 접속 제한으로 1분 간격으로 10명씩 matchId 수집하기

count = 0
 
for player in tqdm(player_list):
  gathering_match(player)
  count += 1
 
  if count == 10:
    time.sleep(60)
    count = 0

In [ ]:
# 수집 matchId 확인
print(match_list)
print(len(match_list))

# 중복 제거
match_list = list(set(match_list))
print(len(match_list))

In [ ]:
# matchId 리스트 복제

matches = match_list[:]

In [ ]:
# 매치별 데이터 수집을 위해 비어있는 DataFrame 생성 

mainDB = pd.DataFrame()

In [ ]:
# 매치별 플레이어 정보 수집을 위한 함수

def gathering_data(match):
    
    url = f"https://api.pubg.com/shards/steam/matches/{match}"

    r = requests.get(url, headers=header)

    match_data = json.loads(r.text)
    match_included = match_data['included']

    temp = pd.DataFrame()

    for i in match_included:
      if i['type'] == 'participant':
        sub = i['attributes']['stats']
        main = pd.DataFrame(sub, index = [0])
        temp = temp.append(main)

    return temp # 한 매치에 참여한 모든 플레이어 데이터

In [ ]:
# 매치 별 플레이어 데이터 수집 및 통합 1

for match in tqdm(matches[:10000]):
  mainDB = mainDB.append(gathering_data(match))

In [ ]:
# 매치 별 플레이어 데이터 수집 및 통합 2

for match in tqdm(matches[10000:20000]):
  mainDB = mainDB.append(gathering_data(match))

In [ ]:
# 매치 별 플레이어 데이터 수집 및 통합 3

for match in tqdm(matches[20000:30000]):
  mainDB = mainDB.append(gathering_data(match))

In [ ]:
# 매치 별 플레이어 데이터 수집 및 통합 4

for match in tqdm(matches[30007:]):
  mainDB = mainDB.append(gathering_data(match))

In [ ]:
# 확인

mainDB

In [ ]:
# csv 파일로 저장

mainDB.to_csv('/content/drive/MyDrive/PUBG/pubg_data.csv')

In [ ]:
df = mainDB[:]

In [ ]:
# 데이터 확인

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2078356 entries, 0 to 0
Data columns (total 23 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DBNOs            int64  
 1   assists          int64  
 2   boosts           int64  
 3   damageDealt      float64
 4   deathType        object 
 5   headshotKills    int64  
 6   heals            int64  
 7   killPlace        int64  
 8   killStreaks      int64  
 9   kills            int64  
 10  longestKill      float64
 11  name             object 
 12  playerId         object 
 13  revives          int64  
 14  rideDistance     float64
 15  roadKills        int64  
 16  swimDistance     float64
 17  teamKills        int64  
 18  timeSurvived     float64
 19  vehicleDestroys  int64  
 20  walkDistance     float64
 21  weaponsAcquired  int64  
 22  winPlace         int64  
dtypes: float64(6), int64(14), object(3)
memory usage: 380.6+ MB
